Three Trends:

1. More students pass reading than math
2. Average math scores are low for district schools compared to charter schools
3. Some schools are better on average in both reading and math scores than other schools

# Dependencies

In [1]:
import pandas as pd
students_df = pd.read_csv('students_complete.csv')
schools_df = pd.read_csv('schools_complete.csv')
schools_df = schools_df.rename(columns={'name': 'school'})

# District Summary 

In [2]:
schools = students_df['school'].value_counts().count()
student_names = students_df['name'].value_counts().count()
total_budget = schools_df['budget'].sum()
reading = students_df['reading_score'].mean()
math = students_df['math_score'].mean()

passing_math = students_df.query('math_score>=70').count()
math_score = ((passing_math['math_score'] / students_df['math_score'].count()) * 100).round(2)
passing_reading = students_df.query('reading_score>=70').count()
reading_score = ((passing_reading['reading_score'] / students_df['reading_score'].count()) * 100).round(2)
overall_pass = ((math_score + reading_score) / 2).round(2)

district_summary = pd.DataFrame({"Total Schools": [schools], 
                                 "Total Students": [student_names],
                                 "Total Budget": [total_budget],
                                 "Average Reading Score": [reading],
                                 "Average Math Score": [math],
                                 "% Passing Math": [math_score],
                                 "% Passing Reading": [reading_score],
                                 "% Overall Passing": [overall_pass],
                                })
district_summary

,% Overall Passing,% Passing Math,% Passing Reading,Average Math Score,Average Reading Score,Total Budget,Total Schools,Total Students
0,80.4,74.98,85.81,78.985371,81.87784,24649428,15,32715


# School Summary

In [3]:
name_school = schools_df[['school','type', 'budget']]

total_students = pd.DataFrame(students_df.groupby("school")["name"].count())
total_students.reset_index(inplace=True)
total_students.columns=[["school", "Total Students"]]

average_reading_scores = pd.DataFrame(students_df.groupby("school")["reading_score"].mean())
average_reading_scores.reset_index(inplace=True)
average_reading_scores.columns=[["school", "Average Reading Score"]]
average_reading_scores.head()

average_math_scores = pd.DataFrame(students_df.groupby("school")["math_score"].mean())
average_math_scores.reset_index(inplace=True)
average_math_scores.columns=[["school", "Average Math Score"]]

f_filtered = students_df[(students_df.reading_score >= 70)]
r_filtered = pd.DataFrame(f_filtered.groupby("school")["reading_score"].mean())
r_filtered.reset_index(inplace=True)
r_filtered.columns=[["school", "% Passing Reading"]]

mo_filtered = students_df[(students_df.math_score >= 70)]
m_filtered = pd.DataFrame(mo_filtered.groupby("school")["math_score"].mean())
m_filtered.reset_index(inplace=True)
m_filtered.columns=[["school", "% Passing Math"]]
m_filtered.head(5)

merged_df = pd.merge(name_school, total_students, on="school").merge(average_math_scores,on='school').merge(average_reading_scores, on='school').merge(r_filtered, on='school').merge(m_filtered, on='school')
merged_df["Budget Per Student"] = merged_df["budget"] / merged_df["Total Students"]
merged_df["Overall Passing"] = ((merged_df["% Passing Reading"] + merged_df["% Passing Math"]) / 2).round(2)
merged_df = merged_df.iloc[:,[0,1,3,2,8,5,4,6,7,9]]
new = merged_df.rename(columns={'school': 'School','type':'School Type','budget':'Total School Budget','Budget Per Student':'Per Student Budget','Overall Passing':'% Overall Passing Rate'})
new.set_index('School', inplace=True)
new.sort_index(inplace=True)
del new.index.name
new.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Overall Passing Rate
Bailey High School,District,4976,3124928,628.0,81.033963,77.048432,84.362521,84.505124,84.43
Cabrera High School,Charter,1858,1081356,582.0,83.975780,83.061895,84.432612,83.972556,84.20
Figueroa High School,District,2949,1884411,639.0,81.158020,76.711767,84.767745,84.310894,84.54
Ford High School,District,2739,1763916,644.0,80.746258,77.102592,84.612799,84.165687,84.39
Griffin High School,Charter,1468,917500,625.0,83.816757,83.351499,84.253156,84.394602,84.32


# Top performing Schools (By Passing Rate)

In [4]:
highest_passing = new.sort_values("% Overall Passing Rate", ascending=False)
highest_passing.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Overall Passing Rate
Holden High School,Charter,427,248087,581.0,83.814988,83.803279,84.391727,85.040506,84.72
Hernandez High School,District,4635,3022020,652.0,80.934412,77.289752,84.483725,84.936975,84.71
Pena High School,Charter,962,585858,609.0,84.044699,83.839917,84.680390,84.719780,84.70
Wright High School,Charter,1800,1049400,583.0,83.955000,83.682222,84.479586,84.758929,84.62
Johnson High School,District,4761,3094650,650.0,80.966394,77.072464,84.430566,84.742448,84.59


# Bottom Performing Schools (By Passing Rate)

In [5]:
lowest_passing = new.sort_values("% Overall Passing Rate")
lowest_passing.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Overall Passing Rate
Cabrera High School,Charter,1858,1081356,582.0,83.975780,83.061895,84.432612,83.972556,84.20
Griffin High School,Charter,1468,917500,625.0,83.816757,83.351499,84.253156,84.394602,84.32
Shelton High School,Charter,1761,1056600,600.0,83.725724,83.359455,84.362559,84.326679,84.34
Rodriguez High School,District,3999,2547363,637.0,80.744686,76.842711,84.374377,84.339111,84.36
Thomas High School,Charter,1635,1043130,638.0,83.848930,83.418349,84.259585,84.497705,84.38


# Math Scores by Grade

In [6]:
schoolgrouped_df = schools_df[['school']]

math_grade_df = students_df[['grade', 'school', 'math_score']]

m_grade_9th = math_grade_df.loc[math_grade_df["grade"] == "9th"]
m_ren_9 = m_grade_9th.rename(columns={'math_score': '9th'})
del m_ren_9['grade']
m_group_ren_9 = m_ren_9.groupby("school")["9th"].mean()

m_grade_10th = math_grade_df.loc[math_grade_df["grade"] == "10th"]
m_ren_10 = m_grade_10th.rename(columns={'math_score': '10th'})
del m_ren_10['grade']
m_group_ren_10 = m_ren_10.groupby("school")["10th"].mean()

m_grade_11th = math_grade_df.loc[math_grade_df["grade"] == "11th"]
m_ren_11 = m_grade_11th.rename(columns={'math_score': '11th'})
del m_ren_11['grade']
m_group_ren_11 = m_ren_11.groupby("school")["11th"].mean()

m_grade_12th = math_grade_df.loc[math_grade_df["grade"] == "12th"]
m_ren_12 = m_grade_12th.rename(columns={'math_score': '12th'})
del m_ren_12['grade']
m_group_ren_12 = m_ren_12.groupby("school")["12th"].mean()

m_grade_9th_merged = pd.DataFrame.join(schoolgrouped_df, m_group_ren_9, on="school")
m_grade_10th_merged = pd.DataFrame.join(m_grade_9th_merged, m_group_ren_10, on="school")
m_grade_11th_merged = pd.DataFrame.join(m_grade_10th_merged, m_group_ren_11, on="school")
m_grade_12th_merged = pd.DataFrame.join(m_grade_11th_merged, m_group_ren_12, on="school")
m_grade_12th_merged.set_index('school', inplace=True)
m_grade_12th_merged.sort_index(inplace=True)
del m_grade_12th_merged.index.name
m_grade_12th_merged.head()

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164


# Reading Score by Grade

In [7]:
grade_df = students_df[['grade', 'school', 'reading_score']]

grade_9th = grade_df.loc[grade_df["grade"] == "9th"]
ren_9 = grade_9th.rename(columns={'reading_score': '9th'})
del ren_9['grade']
group_ren_9 = ren_9.groupby("school")["9th"].mean()

grade_10th = grade_df.loc[grade_df["grade"] == "10th"]
ren_10 = grade_10th.rename(columns={'reading_score': '10th'})
del ren_10['grade']
group_ren_10 = ren_10.groupby("school")["10th"].mean()

grade_11th = grade_df.loc[grade_df["grade"] == "11th"]
ren_11 = grade_11th.rename(columns={'reading_score': '11th'})
del ren_11['grade']
group_ren_11 = ren_11.groupby("school")["11th"].mean()

grade_12th = grade_df.loc[grade_df["grade"] == "12th"]
ren_12 = grade_12th.rename(columns={'reading_score': '12th'})
del ren_12['grade']
group_ren_12 = ren_12.groupby("school")["12th"].mean()

grade_9th_merged = pd.DataFrame.join(schoolgrouped_df, group_ren_9, on="school")
grade_10th_merged = pd.DataFrame.join(grade_9th_merged, group_ren_10, on="school")
grade_11th_merged = pd.DataFrame.join(grade_10th_merged, group_ren_11, on="school")
grade_12th_merged = pd.DataFrame.join(grade_11th_merged, group_ren_12, on="school")
grade_12th_merged.set_index('school', inplace=True)
grade_12th_merged.sort_index(inplace=True)
del grade_12th_merged.index.name
grade_12th_merged.head(5)

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699


# Scores by School Spending

In [8]:
sony_df = pd.merge(name_school, total_students, on="school").merge(average_math_scores,on='school').merge(average_reading_scores, on='school').merge(r_filtered, on='school').merge(m_filtered, on='school')
sony_df["Budget Per Student"] = sony_df["budget"] / sony_df["Total Students"]
sony_df["Overall Passing"] = ((sony_df["% Passing Reading"] + sony_df["% Passing Math"]) / 2).round(2)
bins = [0, 585, 615, 645, 675]
group_names = ['<$585', '$585-615', '$615-645', '$645-675']
sony_df["Spending Ranges Per Student"] = pd.cut(sony_df["Budget Per Student"],bins,labels=group_names)
sony_group = sony_df.groupby("Spending Ranges Per Student").mean()
del sony_group['budget']
del sony_group['Total Students']
del sony_group['Budget Per Student']
sony_group.head()

,Average Math Score,Average Reading Score,% Passing Reading,% Passing Math,Overall Passing
Spending Ranges Per Student,,,,,
<$585,83.455399,83.933814,84.457674,84.504010,84.482500
$585-615,83.599686,83.885211,84.521475,84.523229,84.520000
$615-645,79.079225,81.891436,84.438364,84.368854,84.403333
$645-675,76.997210,81.027843,84.535230,84.639836,84.590000


# Scores by School Size

In [9]:
name2_school = schools_df[['school','type', 'size']]
sony2_df = pd.merge(name2_school, total_students, on="school").merge(average_math_scores,on='school').merge(average_reading_scores, on='school').merge(r_filtered, on='school').merge(m_filtered, on='school')
sony2_df["Overall Passing"] = ((sony2_df["% Passing Reading"] + sony2_df["% Passing Math"]) / 2).round(2)
bins2 = [0, 1000, 2000, 5000]
group_names2 = ['Small(<1000)', 'Medium(1000-2000)','Large(2000-5000)']
sony2_df["School Size"] = pd.cut(sony2_df["Total Students"],bins2,labels=group_names2)
sony2_group = sony2_df.groupby("School Size").mean()
del sony2_group['size']
sony2_group.head()

,Total Students,Average Math Score,Average Reading Score,% Passing Reading,% Passing Math,Overall Passing
School Size,,,,,,
Small(<1000),694.500,83.821598,83.929843,84.536059,84.880143,84.710
Medium(1000-2000),1704.400,83.374684,83.864438,84.357500,84.390094,84.372
Large(2000-5000),3657.375,77.746417,81.344493,84.531238,84.435547,84.485


# Scores by School Type

In [10]:
name3_school = schools_df[['school','type', 'size']]
sony3_df = pd.merge(name2_school, total_students, on="school").merge(average_math_scores,on='school').merge(average_reading_scores, on='school').merge(r_filtered, on='school').merge(m_filtered, on='school')
sony3_df["Overall Passing"] = ((sony3_df["% Passing Reading"] + sony3_df["% Passing Math"]) / 2).round(2)
sony3_group = sony3_df.groupby("type").mean()
sony3_group.index.names = ['School Type']
sony3_group.head()

,size,Total Students,Average Math Score,Average Reading Score,% Passing Reading,% Passing Math,Overall Passing
School Type,,,,,,,
Charter,1524.250000,1524.250000,83.473852,83.896421,84.423298,84.494351,84.458750
District,3853.714286,3853.714286,76.956733,80.966636,84.531876,84.462903,84.498571
